# Aufbereiten der Daten für den Kreis 11

In [57]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!pip install geopy  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install requests --upgrade
!pip install urllib3 --upgrade

Requirement already up-to-date: requests in /opt/conda/lib/python3.6/site-packages
Requirement already up-to-date: certifi>=2017.4.17 in /opt/conda/lib/python3.6/site-packages (from requests)
Requirement already up-to-date: idna<4,>=2.5; python_version >= "3" in /opt/conda/lib/python3.6/site-packages (from requests)
Requirement already up-to-date: charset-normalizer~=2.0.0; python_version >= "3" in /opt/conda/lib/python3.6/site-packages (from requests)
Requirement already up-to-date: urllib3<1.27,>=1.21.1 in /opt/conda/lib/python3.6/site-packages (from requests)
You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: urllib3 in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# 1 Datensatz Verkehrsunfälle

## 1.1 Datensatz einlesen und reduzieren auf Kreis 11 Daten

In [58]:
df_unfaelle = pd.read_csv('./data/verkehrsunfaelle_2020_mit_quartier_kreis.csv')
df_unfaelle.shape

(5291, 27)

In [59]:
#df_unfaelle = df_unfaelle.loc[df_unfaelle['Kreis'] == 'Kreis 11',]
df_unfaelle.drop(df_unfaelle[df_unfaelle.Kreis != 'Kreis 11'].index, inplace=True)
df_unfaelle.head()

,AccidentUID,Quartier,AccidentType,AccidentType_de,AccidentSeverityCategory,AccidentSeverityCategory_de,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_de,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,CantonCode,MunicipalityCode,AccidentYear,AccidentMonth,AccidentMonth_de,AccidentWeekDay,AccidentWeekDay_de,AccidentHour,AccidentHour_text,AccidentLocation_WGS84_E,AccidentLocation_WGS84_N,Kreis,Jahreszeit,ZeitKategorie
14,9BFFA15CC2414409E05329B9D80A8565,8046,at4,Einbiegeunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2680556,1252485,ZH,261,2020,1,Januar,aw405,Freitag,13.0,13h-14h,47.418246,8.506139,Kreis 11,Winter,Nachmittag
22,9C02A2D8710302EDE05329B9D80AC5D9,8057,at0,Schleuder- oder Selbstunfall,as4,Unfall mit Sachschaden,False,False,False,rt432,Hauptstrasse,2683803,1250936,ZH,261,2020,1,Januar,aw406,Samstag,13.0,13h-14h,47.403910,8.548872,Kreis 11,Winter,Nachmittag
38,9DA9A08B38DAAD65E05329B9D80A97B1,8050,at7,Parkierunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2683541,1251753,ZH,261,2020,1,Januar,aw401,Montag,7.0,07h-08h,47.411291,8.545554,Kreis 11,Winter,Rushhour_Morgen
67,A4CEB8229AD1E67EE05328B9D80A8A77,8050,at3,Abbiegeunfall,as4,Unfall mit Sachschaden,False,False,False,rt432,Hauptstrasse,2683881,1251129,ZH,261,2020,1,Januar,aw403,Mittwoch,19.0,19h-20h,47.405636,8.549941,Kreis 11,Winter,Rushhour_Abend
69,9BAF6D9E8FBAF4F0E05329B9D80AD7E9,8050,at0,Schleuder- oder Selbstunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2684282,1251091,ZH,261,2020,1,Januar,aw403,Mittwoch,20.0,20h-21h,47.405243,8.555246,Kreis 11,Winter,Abend


In [60]:
df_unfaelle.shape

(800, 27)

## 1.2 Reverse Geocoding, um die Quartiere zu bekommen

Dieser Schritt wurde absichtlich nicht schon im ersten Schritt (1_Data_Load_Cleanse) durchgeführt, damit nicht unnötig viel Zeit verwendet wird, um alle Quartiere herauszusuchen für Unfälle, die gar nicht im Kreis 11 stattgefunden haben. Dies ist deshalb wichtig, weil dieser Schritt viel Zeit benötigt.

Im Notebook 1_Data_Load_Cleanse wird deshalb nur für jeden Unfall der Kreis herausgesucht und nicht auch noch das Quartier. (Nachträglich ist mir allerdings klar geworden, dass man auch einfach beide in einem Schritt machen könnte und dafür gar nicht zwei Zugriffe benötigt.)

In [61]:
# mail von Maurizio, 24.11.21, workaround for SSL Error:
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

/opt/conda/lib/python3.6/site-packages/certifi/cacert.pem


In [62]:
# define locator for reverse geocoding
locator = Nominatim(user_agent="Kreis 11 Preprocessing")

/opt/conda/lib/python3.6/site-packages/geopy/adapters.py:639: UserWarning: urllib3 prior to 1.24.2 is known to have a bug with custom ssl contexts: it attempts to load system certificates to them. Please consider upgrading `requests` and `urllib3` packages. See https://github.com/urllib3/urllib3/pull/1566
  UserWarning,


In [63]:
# mögliche Förmate der location:
# 381, Wehntalerstrasse, Affoltern, Kreis 11, Zürich, Bezirk Zürich, Zürich, 8046, Schweiz/Suisse/Svizzera/Svizra Affoltern
# Burger King, 378, Hofwiesenstrasse, Oerlikon, Kreis 11, Zürich, Bezirk Zürich, Zürich, 8050, Schweiz/Suisse/Svizzera/Svizra Oerlikon
# A1;A4, Affoltern, Kreis 11, Zürich, Bezirk Zürich, Zürich, 8046, Schweiz/Suisse/Svizzera/Svizra Affoltern
#
# Nicht alle Adressen haben gleich viele Spalten! typisches Format ist die erste Zeile, kann aber auch aussehen wie Zeile 2 oder 3
# deshalb muss die Angabe zum Quartier von hinten her indiziert werden, also mit [-7]

def get_quartier(coordinates, attempt=1, max_attempts=5):
    try:
        location = locator.reverse(coordinates)
        return location.address.split(",")[-7].strip()
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            print("geocoder timed out... attempt:", attempt )
            return get_quartier(coordinates, attempt=attempt+1)
        raise

In [64]:
# Achtung läuft lange! Nicht einfach so laufen lassen... 

quartiere=[]

for ind, row in df_unfaelle.iterrows():
    if ind % 100 == 0:
        print("am Arbeiten... aktueller Index:", ind)      # einfach um zu wissen ob er noch läuft... (mega langsam!!)
    coordinates = [row["AccidentLocation_WGS84_E"],row["AccidentLocation_WGS84_N"]]
    quartiere.append([ind,get_quartier(coordinates)])      # index wird mit abgespeichert, um problematische Datensätze 
                                                             # einfacher wieder finden n
print("alle Koordinaten fertig bearbeitet")

am Arbeiten... aktueller Index: 1400
am Arbeiten... aktueller Index: 1600
am Arbeiten... aktueller Index: 2300
am Arbeiten... aktueller Index: 4500
am Arbeiten... aktueller Index: 4900
alle Koordinaten fertig bearbeitet


### 1.2.1 Überprüfen auf fehlerhafte Quartiernamen und diese manuell korrigieren

In [65]:
# überprüfen, ob für alle Unfälle ein korrektes Quartier gefunden ist
df_quartiere=pd.DataFrame(quartiere, columns=['index','quartier'])
df_quartiere['quartier'].value_counts().to_frame()

,quartier
Oerlikon,279
Affoltern,278
Seebach,241
Birchstrasse,2


In [66]:
df_quartiere.loc[df_quartiere['quartier']=='Birchstrasse']

,index,quartier
402,2631,Birchstrasse
485,3222,Birchstrasse


In [67]:
location = locator.reverse(df_unfaelle.loc[2631,["AccidentLocation_WGS84_E","AccidentLocation_WGS84_N"]])
location

Location(Birchstrasse, Seebach, Kreis 11, Zürich, Bezirk Zürich, Zürich, Schweiz/Suisse/Svizzera/Svizra, (47.4304627, 8.549845044005329, 0.0))

In [68]:
location = locator.reverse(df_unfaelle.loc[3222,["AccidentLocation_WGS84_E","AccidentLocation_WGS84_N"]])
location

Location(Birchstrasse, Seebach, Kreis 11, Zürich, Bezirk Zürich, Zürich, Schweiz/Suisse/Svizzera/Svizra, (47.4304627, 8.549845044005329, 0.0))

In [69]:
df_quartiere = df_quartiere.replace('Birchstrasse','Seebach')

In [70]:
#df_quartiere[df_quartiere['index'].isin([2631,3222])]
df_quartiere['quartier'].value_counts().to_frame()

,quartier
Oerlikon,279
Affoltern,278
Seebach,243


In [71]:
df_unfaelle = df_unfaelle.reset_index()    
df_unfaelle['Quartier'] = df_quartiere['quartier']
df_unfaelle.head()

,index,AccidentUID,Quartier,AccidentType,AccidentType_de,AccidentSeverityCategory,AccidentSeverityCategory_de,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_de,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,CantonCode,MunicipalityCode,AccidentYear,AccidentMonth,AccidentMonth_de,AccidentWeekDay,AccidentWeekDay_de,AccidentHour,AccidentHour_text,AccidentLocation_WGS84_E,AccidentLocation_WGS84_N,Kreis,Jahreszeit,ZeitKategorie
0,14,9BFFA15CC2414409E05329B9D80A8565,Affoltern,at4,Einbiegeunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2680556,1252485,ZH,261,2020,1,Januar,aw405,Freitag,13.0,13h-14h,47.418246,8.506139,Kreis 11,Winter,Nachmittag
1,22,9C02A2D8710302EDE05329B9D80AC5D9,Oerlikon,at0,Schleuder- oder Selbstunfall,as4,Unfall mit Sachschaden,False,False,False,rt432,Hauptstrasse,2683803,1250936,ZH,261,2020,1,Januar,aw406,Samstag,13.0,13h-14h,47.403910,8.548872,Kreis 11,Winter,Nachmittag
2,38,9DA9A08B38DAAD65E05329B9D80A97B1,Oerlikon,at7,Parkierunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2683541,1251753,ZH,261,2020,1,Januar,aw401,Montag,7.0,07h-08h,47.411291,8.545554,Kreis 11,Winter,Rushhour_Morgen
3,67,A4CEB8229AD1E67EE05328B9D80A8A77,Oerlikon,at3,Abbiegeunfall,as4,Unfall mit Sachschaden,False,False,False,rt432,Hauptstrasse,2683881,1251129,ZH,261,2020,1,Januar,aw403,Mittwoch,19.0,19h-20h,47.405636,8.549941,Kreis 11,Winter,Rushhour_Abend
4,69,9BAF6D9E8FBAF4F0E05329B9D80AD7E9,Oerlikon,at0,Schleuder- oder Selbstunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2684282,1251091,ZH,261,2020,1,Januar,aw403,Mittwoch,20.0,20h-21h,47.405243,8.555246,Kreis 11,Winter,Abend


## 1.3 Daten in einer CSV Datei abspeichern

In [72]:
df_unfaelle.to_csv('./data/Verkehrsunfaelle_2020_kreis11.csv', index=False)

# 2 Datensatz Verkehrszählung

## 2.1 Datensatz einlesen und reduzieren auf Kreis 11 Daten

In [73]:
df_traffic = pd.read_csv('./data/verkehrszaehlung_2020_mit_quartier_kreis.csv', low_memory=False)

In [74]:
df_traffic.shape

(399672, 36)

In [75]:
df_traffic.drop(df_traffic[df_traffic.Kreis != 'Kreis 11'].index, inplace=True)
df_traffic.head()

,Unnamed: 0,MSID,MessungDatum,ZeitKategorie,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungZeit,Jahr,Monat,Tag,Wochentag,MonatText,Jahreszeit,Koord_WGS84_E,Koord_WGS84_N,Kreis,Quartier,Verkehrsaufkommen
39528,39528,Z010M001,2020-01-01,Nacht,Unbekannt,Z010,Wehntalerstrasse (Zehntenhausplatz),Wehntalerstrasse,Unbekannt,Holderbachweg,2680425.14,1252660.75,auswärts,235,Wehntalerstr 566 (Fg-Uebergang Schulhaus Holde...,2,11,12,Unbekannt,Unbekannt,2020-01-01 00:00:00,2021-02-04,1695.0,Gemessen,00:00:00,2020,1,2,Mittwoch,Januar,Winter,47.419842,8.504437,Kreis 11,Affoltern,Mittel
39529,39529,Z010M001,2020-01-01,Rushhour_Morgen,Unbekannt,Z010,Wehntalerstrasse (Zehntenhausplatz),Wehntalerstrasse,Unbekannt,Holderbachweg,2680425.14,1252660.75,auswärts,235,Wehntalerstr 566 (Fg-Uebergang Schulhaus Holde...,2,11,12,Unbekannt,Unbekannt,2020-01-01 06:00:00,2021-02-04,391.0,Gemessen,06:00:00,2020,1,2,Mittwoch,Januar,Winter,47.419842,8.504437,Kreis 11,Affoltern,Gering
39530,39530,Z010M001,2020-01-01,Morgen,Unbekannt,Z010,Wehntalerstrasse (Zehntenhausplatz),Wehntalerstrasse,Unbekannt,Holderbachweg,2680425.14,1252660.75,auswärts,235,Wehntalerstr 566 (Fg-Uebergang Schulhaus Holde...,2,11,12,Unbekannt,Unbekannt,2020-01-01 10:00:00,2021-02-04,699.0,Gemessen,10:00:00,2020,1,2,Mittwoch,Januar,Winter,47.419842,8.504437,Kreis 11,Affoltern,Gering
39531,39531,Z010M001,2020-01-01,Nachmittag,Unbekannt,Z010,Wehntalerstrasse (Zehntenhausplatz),Wehntalerstrasse,Unbekannt,Holderbachweg,2680425.14,1252660.75,auswärts,235,Wehntalerstr 566 (Fg-Uebergang Schulhaus Holde...,2,11,12,Unbekannt,Unbekannt,2020-01-01 13:00:00,2021-02-04,1046.0,Gemessen,13:00:00,2020,1,2,Mittwoch,Januar,Winter,47.419842,8.504437,Kreis 11,Affoltern,Gering
39532,39532,Z010M001,2020-01-01,Rushhour_Abend,Unbekannt,Z010,Wehntalerstrasse (Zehntenhausplatz),Wehntalerstrasse,Unbekannt,Holderbachweg,2680425.14,1252660.75,auswärts,235,Wehntalerstr 566 (Fg-Uebergang Schulhaus Holde...,2,11,12,Unbekannt,Unbekannt,2020-01-01 16:00:00,2021-02-04,1475.0,Gemessen,16:00:00,2020,1,2,Mittwoch,Januar,Winter,47.419842,8.504437,Kreis 11,Affoltern,Gering


In [76]:
df_traffic.shape

(43920, 36)

In [77]:
df_traffic.ZeitKategorie.unique()

array(['Nacht', 'Rushhour_Morgen', 'Morgen', 'Nachmittag',
       'Rushhour_Abend', 'Abend'], dtype=object)

Ein Reverse Geocoding ist für diese Daten nicht mehr nötig, da dies schon im ersten Schritt (1_Data_Load_Cleanse) gemacht worden ist. Dies war dort möglich, da es auf dem ganzen Stadtgebiet nur 94 Zählstellen gibt. Für die Unfalldaten wurde dies dort nicht gemacht, weil dies mit den Unfalldaten für alle Kreise sehr lange dauern würde.

## 2.2 Daten in einer CSV Datei abspeichern

In [78]:
df_traffic.to_csv('./data/Verkehrszaehlung_2020_kreis11.csv', index=False)